In [ ]:
import re


class RegularExpressionGenerator:
    def __init__(self):
        self.q = []
        self.sigma = []
        self.delta = {}
        self.q0 = ''
        self.f = []
        self.epsilon = ''

    def initialize(self, q, sigma, delta, q0, f, epsilon):
        self.q = q
        self.sigma = sigma
        self.delta = delta
        self.q0 = q0
        self.f = f
        self.epsilon = epsilon

    def load_from_file(self, path):
        with open(path) as f:
            lines = f.readlines()

        q = lines[0].strip().split(',')
        sigma = lines[1].strip().split(',')
        delta = {}
        for line in lines[2:-2]:
            parts = line.strip().split(',')
            q1, s, q2_list = parts[0], parts[1], parts[2:]
            for q2 in q2_list:
                if (q1, s) not in delta:
                    delta[(q1, s)] = [q2]
                else:
                    delta[(q1, s)].append(q2)
        q0 = lines[-2].strip()
        f = lines[-1].strip().split(',')
        epsilon = ''

        self.initialize(q, sigma, delta, q0, f, epsilon)

    def empty_transitions(self, state):
        frontier = [state]
        visited = set()
        while frontier:
            node = frontier.pop()
            visited.add(node)
            if (node, self.epsilon) in self.delta:
                for neighbor in self.delta[(node, self.epsilon)]:
                    if neighbor not in visited and neighbor not in frontier:
                        frontier.append(neighbor)
        return visited

    def convert_automaton(self):
        # Add a new start state and connect it to the old start state using an epsilon transition
        new_q0 = 'q0_new'
        self.q.append(new_q0)
        self.delta[(new_q0, self.epsilon)] = [self.q0]
        self.q0 = new_q0

        # Add a new accepting state and connect all old accepting states to it using epsilon transitions
        new_f = 'qf_new'
        self.q.append(new_f)
        for f in self.f:
            if (f, self.epsilon) not in self.delta:
                self.delta[(f, self.epsilon)] = [new_f]
            else:
                self.delta[(f, self.epsilon)].append(new_f)
            self.f = [new_f]

        # Compute the regular expression for each pair of states
        n = len(self.q)
        R = [[[''] * n for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                if i == j:
                    R[i][j] = ['1']
                elif (self.q[i], self.sigma[j]) in self.delta:
                    R[i][j] = self.delta[(self.q[i], self.sigma[j])]
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    R[i][j] = list(set(R[i][j] + [r1 + r2 for r1 in R[i][k] for r2 in R[k][j]]))

        # Return the regular expression for the start and accepting states
        return R[self.q.index(self.q0)][self.q]
